# Linear averaging dynamics
In the first part of the lab we study linear averaging dynamics on graphs.

Let $G=(V,E,W)$ be a weighted graph, and $x(t) \in \mathrm{R}^{V}$ denote the state of the nodes of the graph.

The dynamics of $x(t)$ reads

$$
x(t+1) = Px(t),
$$

where $P$ is the normalized adjacency matrix.
Among the applications, the most popular is opinion dynamics, where $x_i$ indicates the opinion of node $i$. This dynamics is known as French - De Groot.

Note that we assume by convention that the opinion of node $i$ is influenced by the opinion of node $j$ if $P_{ij}>0$, i.e., the link $(i,j)$ has to be interpreted as $i$ watching $j$ and updating her opinion based on opinion of $j$.

**Observation**: observe that $\mathbf{1}$ is an equilibrium distribution, since $\mathbf{1} = P \mathbf{1}$ ($P$ is row-stochastic by construction), i.e., consensus distributions are equilibria of the dynamics.

**Questions**: 
- what are the conditions under which the dynamics converges to an equilibrium?
- what are the conditions under which all equilibria are consensus configurations?

**Theorem**: assume that
- its condensation graph has 1 sink;
- the graph is aperiodic.

Then,

$$
\lim_{t \to +\infty} x(t) = \alpha \mathbf{1},
$$

i.e., the agents get to consensus.

## Wisdom of crowds
Consider a graph, and assume that state of each node represents a noisy estimate of the real state $\mu$, i.e.,

$$
x_i = \mu + y_i,
$$

with $E[y_i]=0$, and the variance $\sigma^2 (y_i) = \sigma^2$ for each $i$.

Assume that the graph is connected and aperiodic Eventually, the agents will reach consensus, i.e., $\lim_{t \to +\infty} x(t) = \alpha \mathbf{1}$. 

**Question**: what is the consensus value $\alpha$?

Notice that $\alpha = \pi' (\mu \mathbf{1} + y) = \mu + \pi'y$, then

$$
E[\alpha] = \mu + \pi' E[y] = \mu
$$

so the final estimate of the network is unbiased. Moreover,

$$
\quad \sigma_{\alpha}^2 = \sigma^2 \sum_{i} \pi_i^2 < \sigma^2,
$$

because $\sum_{i} \pi_i^2 <1$ unless the graph has a unique sink node. The interesting observation is that the estimate $\alpha$ has a smaller variance than $\sigma$, i.e., the crowd is able to reconstruct a more precise estimate of the real state than the single agents of the graph.

Let us verify this on a complete graph, where $\pi_i = 1/n$, thus $\sigma_\alpha^2 = \sigma^2/n$. 

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

G = nx.complete_graph(100)

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

# start with random initial states and run the dynamics 200 times
# store in alfa_err the consensus values at each run
alfa_err = np.zeros(200)

for i in range(200):
# rand returns uniformly random values in [0,1], thus mu = 1/2
    x = np.random.rand(100)
    for n in range(500):
        x = P @ x
    alfa_err[i] = (1/2 - np.mean(x))*(1/2 - np.mean(x))

print("Variance of the node states:", 1/12)
print("Variance of the consensus state:", np.mean(alfa_err), "\n")

As expected the variance of $\alpha$ is about $1/100$ of the original variance.

Note that $\sum_{i} \pi_i^2$ tends to 1 if one node has almost all the centrality, and is minimal when the nodes have the same centrality (as in the complete graph, or in the cycle graph). This implies that if a graph is more democratic, then the consensus algorithm leads to better estimates of the true state. If a few nodes have all the centralities, the consensus value is less reliable.

Let us see this with a another example.

In [ ]:
G = nx.cycle_graph(10)
G = nx.Graph.to_directed(G)
G.remove_edges_from([(0,1),(0,9)])
G.add_edge(0,0)

nx.draw(G, with_labels=True)

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

plt.show()

In [ ]:
# start with random initial states and run the dynamics
alfa_err = np.zeros(200)

for i in range(200):
# rand returns random values in [0,1], thus \mu = 1/2
    x = np.random.rand(10)
    var = np.var(x)
    for n in range(500):
        x = P @ x
    alfa_err[i] = (1/2 - np.mean(x))*(1/2 - np.mean(x))

print("Expected variance of the node states:", 1/12)
print("Empirical variance of the consensus state:", np.mean(alfa_err), "\n")

In this graph the consensus value is exactly the initial state of node $0$, because the condensation graph has 1 sink only, which is node $0$.

$$
\pi = \delta^{(0)}.
$$

This graph is the opposite of the complete graph, in the sense that the invariant distribution centrality is totally on node $0$. Thus the variance of the consensus state equals the variance of the single node.

Note that this argument holds independently of the size of the graph.

### Application: distributed computation of average

Let the node set describe a set of sensors that are deployed in some regions in order to collect measurements of some quantity of interest (for example, the temperature). 

Assume that these sensors have limited communication and computation capabilities that allow each of them to exchange information only with those other sensors that are close enough in space. 

Let the graph $G = (V, E)$ describe the pattern of closeness among the sensors $i$ and $j$ so that there is an undirected link between node $i$ and node $j$ if they can communicate to each other (possibly using link weights decreasing with distance). Then, one can design a distributed algorithm for computing the average of the sensor's measurements based on the averaging dynamics.

Let $x_i(0)$ be the measurement of each node. 

We are interested in designing an iterative distributed algorithm that allows the nodes to compute

$$
x = \frac{1}{n}\sum_i x_i(0)
$$

**First attempt**: we run a consensus algorithm. Since the graph is undirected, $\pi_i = \frac{w_i}{\sum_j w_j}$. Thus, the algorithm converges to a consensus $\alpha \mathbf{1}$ such that

$$
\alpha = \sum_i \pi_i x_i(0) = \sum_{i} \frac{w_i}{w} x_i(0).
$$

If each edge knows its degree $w_i$, each node can rescale its initial state, i.e., $y_i(0) = \frac{x_i(0)}{w_i}$. The consensus algorithm for the variable $y_i$ thus converges to

$$
\alpha_y = \sum_{i} \frac{w_i}{w} y_i(0) = \frac{1}{w} \sum_{i} x_i(0).
$$

If we assume that each node knows the average degree of the network, thus

$$
x = \alpha_y \frac{w}{n} = \alpha_y \overline{w}.
$$


In [ ]:
G = nx.karate_club_graph()

# Fix node positions on all pictures according to spring layout
pos = nx.spring_layout(G) 
nx.draw_networkx(G, pos)

n_nodes = len(G)

x = np.random.rand(n_nodes)

plt.show()

In [ ]:
# Let us run the consensus algorithm for y

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

y = x/degrees

for t in range(1000):
    y = P @ y

print("Average state:", np.mean(x))
# choose arbitrarly the first node, but all the nodes reach consensus on y
print("Average computed distributively", y[0] * np.sum(degrees) / n_nodes)

The algorithm works! Unfortunately, requiring that each node knows the average degree of the network is not realistic (and not distributed).

However, there exists another way to solve the problem.

**Second attempt**: we run a second averaging dynamics, with initial condition $z_i(0) = \frac{1}{w_i}$.

This converges to

$$
\alpha_z = \lim_{t \to + \infty} z_i(t) = \sum_i z_i(0) \frac{w_i}{w} = \sum_{i} \frac{1}{w} = \frac{1}{\overline{w}}
$$

By combining the two, each node can estimate the average estimate by 

$$
\frac{\lim_{t \to + \infty} y_i(t)}{\lim_{t \to + \infty} z_i(t)} = \frac{\alpha_y}{\alpha_z} = \frac{x}{\overline{w}} \cdot \overline{w} = x.
$$

This method does not require any global knowledge on the network and it is distributed.

In [ ]:
# Let us implement this
z = 1/degrees

for t in range(1000):
    z = P @ z

print("Average state:", np.mean(x))
# choose arbitrarly the first node, but all the nodes reach consensus both on y and z
print("Average computed distributively", y[0] / z[0])

# Linear averaging dynamics with stubborn agents
We study how to simulate the linear averaging dynamics on graphs in presence of stubborn agents.

We focus on the optimal placement problem, which consists of optimally chosing the position of a stubborn node on the graph in order to maximize its influence on the asymptotic outcome of the dynamics.

Let us first summarize the theory in presence of stubborn agents.

We are given a network, where the agents $V$ are divided in regular agents $R$ and stubborn agents $S$. The regular agents update their opinion $x(t)$ according to the standard DeGroot model, while the stubborn agents do not update their opinions, so that $u(t) \equiv u$.

Let $Q=P|_{R \times R}$ and $E=P|_{R \times S}$.

Thus, the dynamics for the regular agents read

$$
x(t+1) = Qx(t) + Eu.
$$

Under some assumptions (see the lecture notes for details) the dynamics converges to 

$$
x^* = (\mathbf{I}-Q)^{-1}Eu.
$$

Note that, in contrast with the dynamics without stubborn nodes:
- the asymptotic state is not a consensus;
- the asymptotic state does not depend on the initial opinions.

### Implementation
We start by implementing the averaging dynamics with stubborn nodes. 

To illustrate this procedure, we will analyse the following example that involves a $3 \times 4$ grid graph $\mathcal G$.

In [ ]:
G = nx.generators.lattice.grid_graph(dim=[3,4])
n_nodes = len(G)
print("Number of nodes:", n_nodes)

# labels of nodes are couples: (column,row)
nx.draw_spectral(G, with_labels=True)

print(G.nodes())

plt.show()

In [ ]:
# Construct a dictionary that maps the label of nodes  
# (from (0,0) to (2,1)) to their index (from 0 to n_nodes-1)
indices = dict()
for i in range(n_nodes):
    indices[list(G.nodes)[i]] = i
print(indices)

In [ ]:
# Number of iterations
n_iter = 50;
    
# Stubborn and regular nodes
stubborn = [(0,0), (3,2)];
stubborn_id = [indices.get(key) for key in stubborn]
regular = [node for node in G.nodes if node not in stubborn]
regular_id = [id for id in range(n_nodes) if id not in stubborn_id]
print("Stubborn nodes:", stubborn, "\n")
print("Regular nodes:", regular, "\n")

In [ ]:
# Input to stubborn nodes
u = [0,1]

# P matrix
A = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
A = A.toarray() # convert A to a numpy array
degrees = np.sum(A,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ A

# Submatrices
# Using ix_ one can construct index arrays that will 
# index a cross product. 
# a[np.ix_([1,3],[2,5])] returns the array [[a[1,2] a[1,5]], [a[3,2] a[3,5]]].
Q = P[np.ix_(regular_id, regular_id)]
E = P[np.ix_(regular_id, stubborn_id)]

# Sample a random initial condition for regular nodes
ic = np.random.uniform(0,1,len(regular))

# Set the initial condition for the dynamics
x = np.zeros((n_nodes,n_iter))
x[stubborn_id,0] = u;
x[regular_id,0] = ic;
print("Initial state:", x[:,0], "\n")

In [ ]:
# Evolve the opinion vector
for t in range(1,n_iter):
    x[regular_id, t] = Q @ x[regular_id, t-1] + E @ x[stubborn_id, t-1]
    x[stubborn_id, t] = x[stubborn_id, t-1];

print("Final state:")
x_final = x[:,n_iter-1]
for key in indices.keys():
    print(key, x_final[indices[key]])

In [ ]:
fig = plt.figure(1, figsize=(7,7))
ax = plt.subplot(111)

for node in range(n_nodes):
    trajectory = x[node,:]
    ax.plot(trajectory, label='node {0:d}'.format(node))
    
ax.legend()

plt.show()

In [ ]:
average = np.average(x_final)
print("Average asymptotic opinion:", average)

As expected, the dynamics does not converge to consensus. Moreover, in contrast with averaging without input stubborn nodes, we can verify that the asymptotic equilibrium does not depend on the initial condition. Furthermore, the dynamics converge to an equilibrium even though the graph is periodic.

In [ ]:
# Sample another random initial condition for regular nodes
ic = np.random.uniform(0,1,len(regular))

x = np.zeros((n_nodes,n_iter))
x[stubborn_id,0] = u;
x[regular_id,0] = ic;
print("Initial state:", x[:,0], "\n")

# Evolve the opinion vector
for t in range(1,n_iter):
    x[regular_id, t] = Q @ x[regular_id, t-1] + E @ x[stubborn_id, t-1]
    x[stubborn_id, t] = x[stubborn_id, t-1];

print("Final state:")
x_final = x[:,n_iter-1]
for key in indices.keys():
    print(key, x_final[indices[key]])
    
fig = plt.figure(1, figsize=(7,7))
ax = plt.subplot(111)

for node in range(n_nodes):
    trajectory = x[node,:]
    ax.plot(trajectory, label='node {0:d}'.format(node))
    
ax.legend()

plt.show()

## Optimal placement of stubborn nodes
Suppose that node $(0,0)$ is stubborn with opinion $u_{(0,0)}=0$. We want to find the optimal position $(i,j)$ of a stubborn node with opinion $1$ in order to maximize the asymptotic average opinion.

A very simple approach is to consider all possible positions $(i,j)$ and pick the best one.

In [ ]:
# Number of iterations
n_iter = 50;

# We will store final opinion vectors and 
# average of final opinions in dictionaries
# where the key is the position (i,j) of the 
# 1-stubborn agent
final_opinions = dict()
average_opinion = dict() 


for (i,j) in G.nodes:
    # Position (0,0) is occupied by the 0-stubborn node
    if (i,j)==(0,0):
        continue
        
    # Stubborn and regular nodes
    stubborn = [(0,0), (i,j)];
    stubborn_id = [indices.get(key) for key in stubborn]
    regular = [node for node in G.nodes if node not in stubborn]
    regular_id = [id for id in range(n_nodes) if id not in stubborn_id]
    print("Stubborn nodes:", stubborn)

    # Input to stubborn nodes
    u = [0,1]


    # P matrix
    A = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
    A = A.toarray() # convert A to a numpy array
    degrees = np.sum(A,axis=1)
    D = np.diag(degrees)
    P = np.linalg.inv(D) @ A

    # Submatrices
    Q = P[np.ix_(regular_id, regular_id)]
    E = P[np.ix_(regular_id, stubborn_id)]

    # Sample a random initial condition for regular nodes
    ic = np.random.uniform(0,1,len(regular))

    # Set the initial condition for the dynamics
    x = np.zeros((n_nodes,n_iter))
    x[stubborn_id,0] = u;
    x[regular_id,0] = ic;

    for t in range(1,n_iter):
        x[regular_id, t] = Q @ x[regular_id, t-1] + E @ x[stubborn_id, t-1]
        x[stubborn_id, t] = x[stubborn_id, t-1];

    final_opinions[(i,j)] = x[:,n_iter-1]
    average_opinion[(i,j)] = np.average(final_opinions[(i,j)])
    print("Average opinion:", average_opinion[(i,j)], "\n")

To visualize the dependence of the average asymptotic opinion on the position of the $1$-stubborn node we can plot the grid graph by setting each node's size and color according to the magnitude of the average asymptotic opinion when the $1$-stubborn is placed in such node.

In [ ]:
# add a dummy (0,0) entry to the dictionary
# to make its size = n_nodes
average_opinion[(0,0)] = 0

plt.figure(1, figsize=(7,3))
nx.draw(G, 
        pos = nx.spectral_layout(G),
        with_labels=True, 
        node_size = [np.exp(10*average_opinion[node]) for node in G.nodes],
        node_color= [average_opinion[node] for node in G.nodes],
        font_size=8,
        # node's colors are on the red scale
        cmap=plt.cm.Reds)

plt.show()

The optimal placements of the 1-stubborn player are the maximizers of the final average opinion:

In [ ]:
# convert the average opinion values from dict_values to numpy array
avg = np.fromiter(average_opinion.values(),dtype=float)

optimal_place = [place for place in average_opinion.keys() if average_opinion[place]==np.max(avg)]
print("Optimal placements:", optimal_place, "\n")

# print the final opinions under optimal placement
opt_final = final_opinions.get(*optimal_place)
print("Final opinions after optimal placement:", opt_final)

In [ ]:
# plot the asymptotic opinions of the nodes when the stubborn is placed in (1,1)

plt.figure(1, figsize=(7,3))
nx.draw(G, 
        pos = nx.spectral_layout(G),
        with_labels=True, 
        node_size = [np.exp(8*opt_final[indices.get(node)]) for node in G.nodes],
        node_color= [opt_final[indices.get(node)] for node in G.nodes],
        font_size=8,
        # node's colors are on the red scale
        cmap=plt.cm.Reds)

plt.show()

### Back to an old example

In [ ]:
G = nx.cycle_graph(10)
G = nx.Graph.to_directed(G)
G.remove_edges_from([(0,1),(0,9)])
G.add_edge(0,0)

nx.draw(G, with_labels=True)

n_nodes = G.number_of_nodes()

# Construct P
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
W = W.toarray() # convert A to a numpy array
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

n_iter = 100
x = np.zeros((10,n_iter))

# set initial condition (1,0,0,0,0,0,0,0,0,0)
x[0,0] = 1

# evolve the states
for t in range(1,n_iter):
    x[:,t] = P @ x[:,t-1]

print("Average final opinions:", np.mean(x[:,n_iter-1]))

plt.show()

In [ ]:
fig = plt.figure(1, figsize=(7,7))
ax = plt.subplot(111)

for node in range(G.number_of_nodes()):
    trajectory = x[node,:]
    ax.plot(trajectory, label='node {0:d}'.format(node))
    
ax.legend()

plt.show()

We can prove that this is equivalent to having node 0 stubborn with opinion 1.

Indeed, note that because of the topology of the graph, the opinion of node 0 is not influenced by anyone. The same dynamics can be obtained by assuming that node 0 is stubborn.

In [ ]:
# Stubborn and regular nodes
stubborn = [0];
regular = [node for node in G.nodes if node not in stubborn]

print("Stubborn nodes:", stubborn, "\n")
print("Regular nodes:", regular, "\n")

# Input to stubborn nodes
u = [1]

# Submatrices
Q = P[np.ix_(regular, regular)]
E = P[np.ix_(regular, stubborn)]

# Set the initial condition for the dynamics
x = np.zeros((n_nodes,n_iter))
x[stubborn,0] = u;
print("Initial condition:", x[:,0], "\n")

# Evolve the opinion vector
for t in range(1,n_iter):
    x[regular, t] = Q @ x[regular, t-1] + E @ x[stubborn, t-1]
    x[stubborn, t] = x[stubborn, t-1];

x_final = x[:,n_iter-1]
print(x_final)

In [ ]:
fig = plt.figure(1, figsize=(7,7))
ax = plt.subplot(111)

for node in range(G.number_of_nodes()):
    trajectory = x[node,:]
    ax.plot(trajectory, label='node {0:d}'.format(node))
    
ax.legend()

plt.show()

Same trajectory as before!

### A more general model: overview on Friedkin-Johnsen model
The French-DeGroot dynamics can be generalized by taking into account that each agents is not completely regular or completely stubborn. The opinion of each agents is in part due to "innate" opinions, and in part due to influence of society.

The following opinion dynamics model is known as Friedkin-Johnsen model.
Here:
- $x_i(t)$ is the opinion of the agent $i$;
- $y_i$ is the innate opinion of agent $i$.
- $\alpha_i$ is its level of stubborness, i.e., the level of confidence in his/her opinion $y_i$.

The dynamics reads:

$$
x_i(t+1) = \alpha_i y_i + (1-\alpha_i) \sum_{j} P_{ij} x_j(t).
$$

If $\alpha = \mathbf{0}$, we get the French-DeGroot model without input.

If $\alpha \in \{0,1\}^{V}$, we get the French-DeGroot model with stubborn nodes.

As the French-DeGroot model with input, also the Friedkin-Johnsen dynamics converges to a non-consensus state, which depends on $\alpha, y$, but not on the initial opinions $x(0)$.

# Markov chains

## A first example: Random Walks on graphs and flow dynamics
In this section we study a first example of discrete time Markov chain, which is the simple random walk on a graph.

### Simple Random Walk

A random walker on a graph $G$ is an agent that starts at the initial time $0$ at some node and at each time moves from the current position to a neighboring one, chosen with uniform probability (this can be generalized).

To learn how to simulate a random walk, here we consider the example of a $n \times n$ chessboard with a single knight on it. 
1. We construct a network $G$ with all knight's possible moves. In this network nodes represent chessboard locations and an edge between two locations is present if the knight is admitted to move from one to another.
2. We implement a simulation of the knight's random walk on the chessboard network $G$.

In [ ]:
from numpy.random import choice, rand 

#### The Knight's Network

Here we define function `GenerateKnightNetwork` that constructs the knight's network. 
It exploits two auxiliary functions, `ApplyLegalMoves` and `isLegalPos`.

In [ ]:
# Determines if the position obtained applying a move is legal,
# i.e. if it is inside the chessboard
def isLegalPos(x,boardSize):
    if x >= 0 and x < boardSize:
        return True
    else:
        return False
    
# Apply the knight's legal moves to the current position to construct
# neighboring nodes in the knight's graph

def ApplyLegalMoves(x,y,boardSize):
    # will store the neighboring nodes
    new_positions = []
    
    # offsets describe the effect of the knight's legal moves
    # on the current position's row and column
    offsets = [(-1,-2),(-1,2),(-2,-1),(-2,1),
                   ( 1,-2),( 1,2),( 2,-1),( 2,1)]
    
    # for each legal move, compute the new position's row and column
    for off in offsets:
        new_x = x + off[0]
        new_y = y + off[1]
        
        # if the new position doesn't exceed the boardsize,
        # accept it as legal
        if isLegalPos(new_x,boardSize) and isLegalPos(new_y,boardSize):
            new_positions.append((new_x,new_y))
         
    return new_positions

# Generates the graph representing the knigth network.
# Return both the graph object G and the pos dictionary
# for drawing G.

def GenerateKnightNetwork(boardSize):
    # undirected graph G will store the knight's network
    G = nx.Graph()
    # when drawing G, the pos dictionary describes the position on 
    # a boardsize x boardsize grid where to place nodes
    pos = {}
    
    # we assign position to nodes
    # we have boardSize x boardSize nodes
    for row in range(boardSize):
        for column in range(boardSize):
            node_id = row + column*boardSize
            # pos[node_id] are the (x,y) coordinates of node node_id 
            # on a square grid of side boardSize
            pos[node_id] = np.array([1.0*row/boardSize, 1.0*column/boardSize])
            
            # compute the (row,column) of neighboring position to the
            # current one, i.e., positions on the chessoboard reachable
            # by applying legal moves
            neigh_pos = ApplyLegalMoves(row, column, boardSize)
            # for each neigbhoring position, compute the id and add
            # a link in G from current position node_id to neigh_id
            for p in neigh_pos:
                neigh_id = p[0] + p[1]*boardSize
                G.add_edge(node_id, neigh_id)
    return G, pos

We are now ready to generate an example of the knight's network, with a specified boardsize.

In [ ]:
boardSize = 8
# use position dictionary returned by the function GenerateKnightNetwork
(G,pos) = GenerateKnightNetwork(boardSize)
nx.draw(G,pos, with_labels=True, node_color='red')

plt.show()

#### Simulate the Random Walk Process
Now that we have the graph $G$ representing the knight's network, we can simulate the knight's random walk on it. 

The walk starts at some given node and it can either terminate after a specified number of steps or when it first returns to the starting node.

At each step, the walker is at some node `xi` and has to decide which node to visit next. 

In this simple version of the random walk, he does it by choosing a neighbor of the current node uniformly at random.
More formally, he looks at row `xi` of the normalized weight matrix $P$ of the graph $G$ and interprets the numbers on that row as the probability of visiting the corresponding nodes next, given that he currently is at `xi`.

To simulate the random walk we define the function `RandomWalk`, which allows to specify the graph $G$ on which the walk takes place, the starting node and the stopping criterion.

**Remark 1**: the `RandomWalk` function is independent on the specific example we are studying. In other words, it allows to simulate any simple random walk on any **unweighted** graph $G$, since it only exploits the general features of the stochastic process at hand.

In [ ]:
# Simulates a random walk on the graph G, starting from node xi.
# if till_first_return = True the random walk stops the first time
# it returns to the starting node xi.
# Otherwise, it goes on for num_steps steps.

def RandomWalk(G, xi, num_steps, till_first_return = False):
    # nodeSeq stores the sequence of visited nodes
    nodeSeq = []
    nodeSeq.append(xi)
    
    # if the walk ends at the first return to xi
    if till_first_return:
        # stores the initial position to check if the 
        # walk returns to it
        x_init = xi
        
        # no upper bound on the number of steps
        while True:
            # compute the next visited node xi by chosing uniformly
            # at random a neighbor of the current node
            xi = choice(G.adj[xi],1)[0]     
            nodeSeq.append(xi)
            
            # check if the walk has returned to the starting node
            # if so, end the walk
            if xi == x_init:
                return nodeSeq
    
    # if the walk ends after num_steps steps
    else:
        for i in range(num_steps):
            xi = choice(G.adj[xi],1)[0]      
            nodeSeq.append(xi)
        return nodeSeq

You can implement RandomWalk with different stopping criteria, or using non-uniform transition probability distributions (for weighted graphs).

As a first experiment, we simulate a simple random walk on $G$ with $10$ steps.

In [ ]:
# Simulate a random walk on G 
nodeSeq = RandomWalk(G, xi=0, num_steps=10, till_first_return=False)
edgeSeq = [(nodeSeq[i-1], nodeSeq[i]) for i in range(1,len(nodeSeq))]

# Draw G and represent the random walk by colouring the edge sequence
# first draw all nodes and links
nx.draw(G, pos)
# then, on the previous picture, add node labels and highlight the edge sequence
nx.draw(G, pos, with_labels=True, edgelist = edgeSeq, edge_color='blue', node_color='red', width=2)

print("Node sequence:", nodeSeq)

plt.show()

In [ ]:
# Simulate a random walk on G that stops at the first return time
# note that if till_first_return = True, 'num_steps' is negligible
nodeSeq = RandomWalk(G, xi=0, num_steps=1, till_first_return=True)
edgeSeq = [(nodeSeq[i-1], nodeSeq[i]) for i in range(1,len(nodeSeq))]

# Draw G and represent the random walk by colouring the edge sequence
# first draw all nodes and links
nx.draw(G, pos)
# then, on the previous picture, add node labels and highlight the edge sequence
nx.draw(G, pos, with_labels=True, edgelist = edgeSeq, edge_color='blue', node_color='red', width=2)

# if the node sequence is not deducible from the plot, you can print the nodeSeq
print("Node sequence:", nodeSeq)

plt.show()

In [ ]:
# Simulate a random walk on G with 100 steps
nodeSeq = RandomWalk(G, xi=0, num_steps=200, till_first_return=False)
edgeSeq = [(nodeSeq[i-1], nodeSeq[i]) for i in range(1,len(nodeSeq))]

# Draw G and represent the random walk by colouring the edge sequence
# first draw all nodes and links
nx.draw(G, pos)
# then, on the previous picture, add node labels and highlight the edge sequence
nx.draw(G, pos, with_labels=True, edgelist = edgeSeq, edge_color='blue', node_color='red', width=2)

plt.show()

Note that even with a larger number of steps some of the nodes may not be visited!

**Question**: do you think that all the nodes have the same probability of being visited?

To answer this question, we need to relate random walks to flow dynamics.

### Random Walks and Linear Flow Dynamics

The random walk and the flow dynamics are deeply connected. Indeed, if we describe the position of the walker on $G$ at time $t$ with a random variable $x(t)$, the variable's probability distribution evolves according to a flow dynamics. Let

$$
\pi_i(t) = \mathbf{P}\{x(t)=i\}.
$$

Then,

$$
\pi(t+1) = P'\pi(t)
$$

or more explicitly

$$
\pi_j(t+1) = \sum_{i \in \mathcal V} P_{ij}\pi_i(t),
$$

Moreover, one can use a random walk to estimate the invariant measure $\pi$ of the graph $G$ (assume here $G$ is strongly connected and aperiodic). Indeed, by the Katz theorem, the fraction of time spent by the walker on each node tends to the node's value in the invariant measure $\pi$ as the length of the walk increases.

The following section shows how to compute empirical frequencies and how to compare them with the inviariant distribution of $G$.

#### Empirical frequencies and invariant distribution
The empirical frequencies are the fractions of total walk time that each node of $G$ is visited in the random walk. 

They can be represented by a histogram as follows.

We simulate random walks starting at each node of $G$, and we keep track of the sequence of visited nodes. 

In [ ]:
# nodeSeq: list to store all random walks
nodeSeq = []

# simulate one random walk for each initial node in G
for xi in range(G.number_of_nodes()):
    # list.extend extends the list by appending all the items from an iterable
    nodeSeq.extend(RandomWalk(G, xi, 100, False))
    
# plt.hist computes and draws the histogram of nodeSeq. We have one bin for each node,
# so each bin width is 1 and the number of observations in each bin equals the number
# of visits to the correspnding node. 
# Since density=True, the return element will be the counts normalized 
# to form a probability density.
h = plt.hist(nodeSeq, bins = G.number_of_nodes(), density=True)

plt.show()

In order to use empirical frequencies to approximate the invariant measure of $G$, we have to construct long random walks. As an example, here we run a random walk with $10000$ steps for each initial condition.

In [ ]:
# nodeSeq: list to store all random walks
nodeSeq = []

# simulate one random walk for each initial node in G
for xi in range(G.number_of_nodes()):
    # list.extend extends the list by appending all the items from an iterable
    nodeSeq.extend(RandomWalk(G, xi, 10000, False))
    
# plt.hist computes and draws the histogram of nodeSeq. We have one bin for each node,
# so each bin width is 1 and the number of observations in each bin equals the number
# of visits to the correspnding node. 
# Since density=True, the return element will be the counts normalized 
# to form a probability density.
h = plt.hist(nodeSeq, bins = G.number_of_nodes(), density=True)

plt.show()

The frequencies are now smoother, and look not uniform, as expected. Let us compare frequencies with the invariant distribution of the graph.

In [ ]:
# Simulate a "long" random walk
nodeSeq = RandomWalk(G, 0, 100000, False)

h = plt.hist(nodeSeq, bins = G.number_of_nodes(), density=True)

# Compute empirical frequencies

frequencies = np.zeros(len(G))
# count the visits to each node
for node in nodeSeq:
    frequencies[node] += 1
# normalize the counts to obtain frequencies
frequencies /= len(nodeSeq)
print("Frequencies:", frequencies, "\n")

plt.show()

In [ ]:
# sort the nodes to compare empirical frequencies with invariant distribution
H = nx.Graph()
H.add_nodes_from(sorted(G.nodes(data=True)))
H.add_edges_from(G.edges(data=True))

print(H.nodes())

In [ ]:
# Compute P matrix
A = nx.adjacency_matrix(H) # -> return type is scipy.sparse.csr_matrix
A = A.toarray() # convert A to a numpy array
degrees = np.sum(A,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ A

# Compute invariant distribution
values,vectors = np.linalg.eig(P.T)
index = np.argmax(values.real)
pi = vectors[:,index].real
pi = pi/np.sum(pi)

print("pi=", pi, "\n")

print("frequencies=", frequencies, "\n")

# Evaluate the approximation error by computing the norm of
# the difference between the empirical frequencies and the 
# invariant measure
error = np.linalg.norm(frequencies-pi)
print("Error", error)

**Observation**: Note how important the invariant distribution is. So far, it appeared in many different contexts:
- centrality measures in social networks;
- consensus value in averaging dynamics;
- asymptotic limit of linear flow dynamics;
- invariant probability distributions in random walks.

### Exercise
Consider the graph shown in the following picture:
![Exgraph](graph1.png)

**Remark 2**: for nodes without out-going links, follow the convention and add a self-loop.

1. For each node of $G$, simulate the simple random walk starting from that node. What do you observe? How can you justify your observations?
2. Study the graph $G$ and compute its invariant measures. Can you see any relation with the behavior of the random walk?
3. Substitute the link (5,1) with the link (1,5) and repeat the analysis performed at point 1. and 2. What topological property of the graph has changed? How does this reflect on the random walk process?

In [ ]:
# TO DO

## Discrete time Markov chains
We already saw an example of discrete time Markov chain: the random walk on the knight's network.

In general, every discrete time Markov chain can be interpreted as a random walk on a weighted directed graph. So, in the general case, transition probabilities from each node to its neighbors are not uniformly distributed.

To better understand this notion, we analyse the following example.

![DMCgraph](discreteMC.png)

1. Construct the directed graph $G$ with weights as shown in the picture.
2. Compute the invariant probability distribution vector by computing the leading eigenvector of $P'$.
3. Simulate a random walk starting from node 1 on the graph for n = 1000, 2000, 5000, 10000 steps. Determine the fraction of the steps the walk has been in each node i. Compare this with the invariant distribution. What do you observe?

**Hint:** you can adopt the function RandomWalk by modifying the choice of the next node to visit according to the fact that the transition probability is not uniform.

4. What happens with your estimate of $\pi$ if you remove node 5 and all links connected to it from the graph and add a self loop of weight 1 to node 6?
5. Compute the expected hitting time $\mathbb{E}_j[T_S]$, $\forall j \in R = \mathcal V \setminus \mathcal S$, for the set $S = \{2, 5\}$ analytically.
6. For every node i, simulate several times a random walk on $G$ that begins in node i and stops when it comes back to it. Use this simulation to estimate the expected return time, $\mathbb{E}_i[T_i^+]$. Compare this estimate with the expected return times obtained analytically from the expected hitting times.

1.

In [ ]:
G = nx.DiGraph()

# add weighted edges
G.add_weighted_edges_from([(1,3,1),(1,4,2),(2,1,1),(3,1,7),(3,6,3),(4,2,1),(4,3,4),(5,5,2),(5,3,1),(6,5,1)])

# define a new graph with sorted nodes
H = nx.DiGraph()
H.add_nodes_from(sorted(G.nodes(data=True)))
H.add_edges_from(G.edges(data=True))

nx.draw(H,with_labels=True)

# Compute P matrix
A = nx.adjacency_matrix(H) # -> return type is scipy.sparse.csr_matrix
A = A.toarray() # convert A to a numpy array
degrees = np.sum(A,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ A

n_nodes = H.number_of_nodes()

print("P:", P)

2-3.

In [ ]:
# TO DO

4. If node 5 is removed, node 6 is a sink node. Hence, all random walks will eventually reach this node so that the estimate of the invariant measure will converge to $\pi =[0, 0, 0, 0, 1]$. You can check this by simulating the random walk on the modified graph.

The interpretation for this is that as the random walks hits 6, it cannot escape. Thus, from there on, it will stay in 6 forever. In the limit of infinite $t$, the fraction of time spent in node 6 tends to 1. This is consistent with the fact that invariant distribution centrality in a graph has support only on nodes that belong to trapping set.

In [ ]:
# TO DO

5. The expected hitting times  $\hat{x}= (\mathbb{E}_i[T_S])_{i \in R}$ for the set $S$ and for all nodes $i \in R = V \setminus S$ can be computed by solving the system of equations

$$
\hat{x} = \mathbf{1} + \hat{P}\hat{x},
$$ 

where $\hat{P}$ is obtained from $P$ (the normalized weight matrix of the graph) by removing the rows and columns corresponding to the nodes in the set $S$.

More explicitly, the expected hitting times can be expressed as

$$
\hat{x} = (I - \hat{P})^{-1} \mathbf{1}
$$

**Remark**: note that $(I - \hat{P})$ is invertible only if $V \setminus S$ has at least a link pointing to $S$. Indeed, if $(I - \hat{P})$ is not invertible, the random walk starting from nodes in $V \setminus S$ cannot hit nodes in $S$, and the hitting times diverge.

Thus, we get:

In [ ]:
# Define the set S and the remaining nodes R
# Subtract -1 because indexes go from 0 to 5 and nodes from 1 to 6
S = [1, 4] # refer to nodes [2,5]
R = [node for node in range(n_nodes) if node not in S]

# Restrict P to R x R to obtain hat(P)
hatP = P[np.ix_(R, R)]

# solve the linear system to obtain hat(x)
# np.linalg.solve solves a linear matrix equation given
# the coefficient matrix and the dependent variable values
hatx = np.linalg.solve((np.identity(n_nodes-2)-hatP),np.ones(n_nodes-2))
# map node to position of node in hatx
map = {0: 0, 2: 1, 3: 2, 5: 3}

# define the hitting times to the set S
# hitting time is 0 if the starting node is in S
hitting_s = np.zeros(n_nodes)
# hitting time is hat(x) for nodes in R
for r in R:
    hitting_s[r] = hatx[map[r]]

print("hitting times:", hitting_s)

Notice that for node 6 the expected hitting time is 1, because its only out-neighbour belongs to $S$.

6. To compute expected return times analytically, recall that they can be caracterized by

$$
\mathbb{E}_i[T_i^+] = 1 + \sum_{j} P_{ij} \mathbb{E}_j[T_i]
$$

where $\mathbb{E}_j[T_i]$ is the expected hitting time to the set $S={i}$ starting from $j$.

So, for computing $\mathbb{E}_i[T_i^+]$, one can:
- set $S=\{i\}$
- compute the expected hitting time to $S$, $$\mathbb{E}_j[T_i], \quad \forall j \in V\setminus \{i\}$$ (as done in point 5)
- apply the linear relation $\mathbb{E}_i[T_i^+] = 1 + \sum_{j} P_{ij} \mathbb{E}_j[T_i]$

Instead, an estimation of the expected return time $\mathbb{E}_i[T_i^+]$, $\forall i$, is obtained by simulating random walks that start at $i$ and end at the first return (you can use `RandomWalk(G, xi=i, num_steps = 'inf', till_first_return = True)`)

Implement and compare the two approaches.

In [ ]:
# TO DO

## Continuous Time Markov Chains

In CTMC, the time is not discrete ($t=0,1,\ldots$) but it flows in a continuum ($ t \geq 0$).

The random process still describes the evolution of a state variable $x$ inside a discrete state space $\mathcal X$ with a graph structure.
We are given a graph $G =(\mathcal X, \Lambda)$ with nodes $\mathcal X$ and weight matrix $\Lambda$ describing possible transitions between nodes/states.

Transitions now happen at random time instants that are decided by the tick of a so called **Poisson clock**. A Poisson clock is characterized by the property that the time elapsed between any two of
its consecutive ticks is an independent random variable with exponential distribution with a specified rate.

**Remark 1**:
to simulate continuous time Markov chains the following fact will be useful.
To simulate a Poisson clock with rate $r$, one must simulate the time between two consecutive ticks, which we denote by $t_{next}$. We can compute $t_{next}$ as

$$ t_{next} = - \frac{\ln(u)}{r}$$

where $u$ is a random variable with uniform distribution, $u \in \mathcal{U}(0,1)$.


### Modelling Continuous time Markov chains
There are three equivalent ways of modelling CTMCs.

**1st approach**
1. you define a unique **global** Poisson clock with an appropriate rate $\omega^* = \max_i(\omega_i)$ where $\omega_i= \sum_j \Lambda_{ij}$
2. when you are at node $i$ and **the global clock ticks**, either you jump to a neighbor $j$ with probability $\bar P_{ij} = \frac{\Lambda_{ij}}{\omega_{*}}, \; i \neq j$ or you stay in the same node (no transition) with probability $\bar P_{ii} = 1 - \sum_{i \neq j} \bar P_{ij}$.

In this approach, the continuous time is discretized using a global clock, while the matrix $\bar P$ describes the jumps. For this reason the matrix $\bar P$ is called **jump chain** of the CTMC.

Notice that $\bar P_{ii}=0$ for the nodes $i$ maximizing $\omega$, and it is larger as $\omega_i/\omega$ is small.

**2nd approach**
1. each node $i$ is equipped with its own Poisson clock with rate $\omega_i= \sum_j \Lambda_{ij}$.
2. when you are at node $i$ and **the clock of that node ticks**, you jump to a neighbor $j$ with probability  $P_{ij} = \frac{\Lambda_{ij}}{\omega_i}$.

**3rd approach**
1. each link $(i,j)$ is equipped with a Poisson clock with rate $\omega_{(i,j)} = \Lambda_{ij}$. 
2. when the clock of link $(i,j)$ ticks and the Markov chain is in $i$, it moves to $j$.

For nodes $i$ such that $\omega_i=0$ (which means that once the process in $i$ it remains $i$ forever), we need to add a selfloop $\Lambda_{ii}>0$, otherwise the matrix $P$ is not well defined.

### Example

Suppose that the weather can be modelled as a continuous-time Markov chain, with state space $\mathcal{X} = \{sunny,rainy, cloudy,snowy\}$. Let the transition rates be

![transitionRates](transitionMatrix.png)

1. The probability distribution $\bar{\pi}(t)$ of the CTMC $X(t)$ with transition rate matrix $\Lambda$ is defined as

$$
\bar{\pi}_i(t) = \mathbb P(X(t) = i), \quad i \in \mathcal X \,.
$$

It evolves according to the equation

$$
\frac{d}{dt} \bar{\pi}(t) = -L'\bar{\pi}(t)
$$

where $L= diag(w) - \Lambda$, with $w = \Lambda \mathbf{1}$.

Thus, the invariant probability vectors are eigenvector of $L'$ corresponding to eigenvalue $0$. It can also be proven that $\bar{\pi}$ is the left dominant eigenvector of $\bar P$, where $\bar P$ is the row-stochastic matrix defined as

$$
\bar P_{ij} = \frac{\Lambda_{ij}}{\omega_{*}}, \; i \neq j \quad \bar P_{ii} = 1 - \sum_{i \neq j} \bar P_{ij}
$$

with $\omega = \Lambda \mathbf{1}$ and $\omega_{*}=\max_i \omega_i$.

Compute the invariant probability vector $\bar{\pi}$ of the CTMC by determining the leading eigenvector of the matrix $\bar P'$

In [ ]:
Lambda = [
[0, 1/30, 1/15, 1/60],
[1/60, 0, 1/10, 1/100],
[1/25, 1/10, 0, 1/50],
[1/100, 1/10, 1/10, 0]]

w = np.sum(Lambda, axis=1)
w_star = np.max(w)
# compute the off-diagonal part of P_bar
P_bar = Lambda/w_star 
# add the diagonal part
P_bar = P_bar + np.diag(np.ones(len(w))-np.sum(P_bar,axis=1))

# compute dominant eigenvector
values,vectors = np.linalg.eig(P_bar.T)
index = np.argmax(values.real)
pi_bar = vectors[:,index].real
pi_bar = pi_bar/np.sum(pi_bar)
print("pi_bar=", pi_bar)

nstates = len(pi_bar)

2. Simulate the continuous-time Markov chain starting from sunny weather. Do this following two different approaches:

**a)** 1st approach, i.e., using global clock with rate $\omega^* = \max_i{\omega_i}$ and the conditional probability matrix $\bar P$.

**b)** 2nd approach, i.e., using a rate-$\omega_i$ clock in each node $i$ and the conditional probability matrix $P$.

In both cases:
- plot the trajectory for the first 20 jumps,
- use the simulation to estimate $\bar{\pi}$.

In [ ]:
# 1st approach: global clock with rate w_star and matrix P_bar

# set the number of steps in the simulation
n_steps = 10000
# pos will keep trace of the visited states
pos = np.zeros(n_steps, dtype=int)
# we start from state 0 (sunny)
pos[0] = 0
# transition_times will store the time instants at which jumps/transitions happen
transition_times = np.zeros(n_steps)
# the random time to wait for the next transition
# is drawn according to its distribution, as discussed in Remark 1
# NOTE: in the formula for t_next we use w_star, the rate of the "global" Poisson clock
t_next = -np.log(np.random.rand())/w_star

for i in range(1,n_steps):
    # the next state to visit will be extracted according to the probabilities
    # stored in the row of P_bar corresponding to the current state.
    # We extract a value pos[i] in (0,...,num_states-1) according to the discrete distribution P_bar[pos[i-1],:]
    pos[i] = np.random.choice(nstates, p=P_bar[pos[i-1],:])
    transition_times[i] = transition_times[i-1] + t_next
    # compute the waiting time to the next transition
    t_next = -np.log(np.random.rand())/w_star

# plot the trajectory for the first 20 jumps
plt.plot(transition_times[0:20], pos[0:20], 'bo')
plt.title('Trajectory for the first 20 jumps')

# Estimate pi

pi_estimate = np.zeros(nstates)
# We have the time instants of all transitions, we now compute time intervals.
# np.diff computes the n-th discrete difference of a vector.
# Here we set n=1 to compute first difference, which is given by 
# intervals[i] = transition_times[i+1] - transition_times[i].
# We also provide a value to append to transition_times prior to performing the difference
# so that we can compute also the last interval: 
# transition_times[-1] + t_next is the end of the time horizon.
intervals = np.diff(transition_times, n=1, append = transition_times[-1] + t_next)

# for each state in the state space
for state in range(nstates):
    # identify the steps when we visited that state during the process
    visits = np.argwhere(pos == state)
    # the estimate of the invariant measure for that state is equal to the
    # time spent on the state divided the total time of the process
    pi_estimate[state] = np.sum(intervals[visits])/(transition_times[-1] + t_next)
    
print("Estimate of pi_bar:", pi_estimate)

plt.show()

In [ ]:
# 2nd approach: local clocks with rates w_i and matrix P

# contruct the P matrix (instead of P_bar) and clock rates w
w = np.sum(Lambda, axis=1)
D = np.diag(w)
P = np.linalg.inv(D) @ Lambda

# set the number of steps in the simulation
n_steps = 10000
# pos will keep trace of the visited states
pos = np.zeros(n_steps, dtype=int)
# we start from state 0
pos[0] = 0
# transition_times will store the time instants at which
# jumps/transitions happen
transition_times = np.zeros(n_steps)
# the random time to wait for the next transition
# is drawn according to its distribution, discussed in Remark 2
# NOTE: in the formula for t_next we use the rate of the clock of the current state, in this case w[0].
t_next = -np.log(np.random.rand())/w[0]

for i in range(1,n_steps):
    # the next state to visit will be extracted according to the probabilities
    # stored in the row of P corresponding to the current state.
    pos[i] = np.random.choice(nstates, p=P[pos[i-1],:])
    # store the time instant of the current transition
    transition_times[i] = transition_times[i-1] + t_next
    # compute the waiting time to the next transition
    # NOTE: we use the rate w[pos[i]] of the clock of the current position
    t_next = -np.log(np.random.rand())/w[pos[i]]

# plot the trajectory for the first 20 jumps
plt.plot(transition_times[0:20], pos[0:20], 'bo')
plt.title('Trajectory for the first 20 jumps')

# Estimate pi

pi_estimate = np.zeros(nstates)

intervals = np.diff(transition_times, n=1, append = transition_times[-1] + t_next)
for node in range(nstates):
    visits = np.argwhere(pos == node)
    pi_estimate[node] = np.sum(intervals[visits])/(transition_times[-1] + t_next)
print("Estimate of pi_bar:", pi_estimate)

plt.show()

Simulating with the third approach (link-based) is more complicated, because it requires to consider multiple Poisson clocks at the same time. Indeed, when $X(t)=i$, one should at each time-step:
1. compute the random tick for each of the Poisson clocks of the links $(i,j)$ with rate $\omega_{(i,j)} = \Lambda_{ij}$
2. select the link $(i,j)$ that ticked first (recall that each tick is random)
3. Update the state of the Markov chain to $j$.

3. An ice-cream shop’s profit depends on the weather. Their profit per time unit is given by the following function

$$
f(X) = \begin{cases} 
10 & \text{if X = sunny}\\
2 & \text{if X = cloudy}\\
1 & \text{if X = rainy}\\
0 & \text{if X = snowy}
\end{cases}
$$

Simulate how the profit grows with time. Compute the average profit, both from the simulation and from the stationary distribution $\bar{\pi}$.

In [ ]:
# Simulation of profit growth
# I choose to simulate the CTMC following the first approach

# set the number of steps in the simulation
n_steps = 10000
# payoff values corresponding to each state
payoff = [10, 2, 1, 0]

pos = np.zeros(n_steps, dtype=int)
pos[0] = 0
transition_times = np.zeros(n_steps)
# since I'm following the first simulation approach, here I divide by the rate w_star of the global clock
t_next = -np.log(np.random.rand())/w_star
# we define a profit variable, which stores the comulative profit up to
# the current time. For the first interval, which is t_next long, 
# the profit grows by payoff[pos[0]]*t_next
profit = payoff[pos[0]]*t_next

# we evolve the process as done before, increasing at each step the total profit
for i in range(1,n_steps):
    pos[i] = np.random.choice(nstates, p=P_bar[pos[i-1],:])
    transition_times[i] = transition_times[i-1] + t_next
    # compute the waiting time to the next transition
    # NOTE: we use the rate w[pos[i]] of the clock of the current position
    t_next = -np.log(np.random.rand())/w_star
    # during the next interval, which is t_next long, the process will be in state
    # pos[i] and the profit will grow by payoff[pos[i]]*t_next
    profit = profit + payoff[pos[i]]*t_next
    
# the average profit is estimated as the overall profit obtained along the
# simulation divided by the total time of the process
average_profit = profit/(transition_times[-1] + t_next)
print("Average profit", average_profit)